In [1]:
import numpy as np
import matplotlib.pyplot as plt
from finite_volume.advection import AdvectionSolver
from finite_volume.initial_conditions import generate_ic

def l1(x: np.ndarray) -> float:
    return np.mean(np.abs(x))


def l2(x: np.ndarray) -> float:
    return np.sqrt(np.mean(np.square(x)))


def linf(x: np.ndarray) -> float:
    return np.max(np.abs(x))

In [2]:
p=1
n_rotations=1
quadrature='gauss-legendre'
ic_type__PAD="square",(0,1)
limiter_config=dict(
    aposteriori_limiting=True, 
    SED=False, 
    hancock=False, 
    convex=False, 
    fallback_to_first_order=False, 
    fallback_limiter='minmod'
    )

In [3]:
ic_type, PAD = ic_type__PAD

def u0(x, y):
    return generate_ic(type=ic_type, x=x, y=y)

def u0_rotated(x, y):
    return np.rot90(u0(x, y), k=n_rotations, axes=(1, 0))

u0_rotated.__name__ += f"_{n_rotations}"

vx_vy = (1, 0)
vx_vy_C4 = {0: (1, 0), 1: (0, 1), 2: (-1, 0), 3: (0, -1)}

shared_config = dict(
    **limiter_config,
    save=False,
    load=False,
    PAD=PAD,
    n=(16,),
    order=p + 1,
    flux_strategy=quadrature,
    courant=0.8,
    snapshot_dt=1,
)

# baseline
solver = AdvectionSolver(
    **shared_config,
    u0=u0,
    v=vx_vy,
)
solver.rkorder()

# reflected
solver_rotated = AdvectionSolver(
    **shared_config,
    u0=u0_rotated,
    v=vx_vy_C4[n_rotations],
)
solver_rotated.rkorder()

# check equivariance
diffs = solver.u_snapshots[-1][1] - np.rot90(
    solver_rotated.u_snapshots[-1][1], k=-n_rotations, axes=(1, 0)
)
# diffs = solver.u_snapshots[-1][1] - np.flipud(np.rot90(np.flipud(solver_rotated.u_snapshots[-1][1]), k=-n_rotations))
print(f"{l1(diffs)=}")
print(f"{l2(diffs)=}")
print(f"{linf(diffs)=}")

New solution instance...


  0%|          | 0/1 [00:00]

100%|██████████| 1.0/1 [00:01]                



New solution instance...


100%|██████████| 1.0/1 [00:00]                


l1(diffs)=0.0
l2(diffs)=0.0
linf(diffs)=0.0
